# Full TensorFlow input and training pipeline for large datasets with data augmentation

## Introduction

In TensorFlow documentation, there are many examples of specific tasks. But I haven't found any full example which could be used as a starting point for my real life Deep Learning projects.

Thus, the purpose of this notebook is to show an example of a full Deep Learning workflow with TensorFlow:

1. Read large training data from files
2. Apply data augmentation
3. Train the model
4. Compute validation loss
5. Send logs to TensorBoard
6. Save and restore model
7. Use trained model for prediction

In TensorFlow, there are often several ways of achieving a specific task. If you have any suggestion to improve this notebook, don't hesitate to tell me!

__Disclaimer__

> This notebook is not a TensorFlow tutorial. It assumes you have a good basic knowledge of TensorFlow mechanics.

### The Dataset API

For reading data, we will use the new high level Dataset API. With this API, we will not need queues.

We will show how to use the Dataset API for:

1. loading multiple files for each input example,
2. data augmentation.

Indeed, loading multiple files for each input example can be useful for several applications:

* dealing with image sequences,
* object detection with multiple cameras,
* etc.

## Let's go!

__Note on tensorflow version:__

> In tensorflow v1.4, *tf.contrib.data* has been integrated into *tf.data*. Please use appropriate import according to your tensorflow version. See comment in code below.

In [1]:
import tensorflow as tf
import numpy as np
import os.path

# For tensorflow 1.3
#from tensorflow.contrib import data as tfdata

# For tensorflow 1.4 and above
from tensorflow import data as tfdata

### Create some dummy data

We are going to create some dummy data files. In a real application, use your own data files. Here, we will create 4 files: two groups of two files. Each file will contain a 3x3 matrix.

In [2]:
height = 3
width = 3

# Create random files containing raw matrix of shape 3x3
for i in range(2):
    for j in range(2):
        # Create matrix with fake data
        matrix = np.zeros((height,width)) + i + (j*10)
        # Save matrix as raw float32 file
        matrix.astype('float32').tofile('data/file_' + str(i) + '_' + str(j) + '.raw')

Let's have a look at one of our dummy data files.

In [3]:
# Print one of the generated files for checking
matrix = np.fromfile('data/file_1_1.raw', dtype=np.float32)
matrix = matrix.reshape((height,width))
print(matrix)

[[ 11.  11.  11.]
 [ 11.  11.  11.]
 [ 11.  11.  11.]]


OK, our dummy data file looks good.

### Create parser

We will need a parser to read our examples from the data files.

Here, each example will be created by stacking data coming from two different files and a label. Therefore, the parser arguments will be the two filenames and the label. It will return two tensors containing the example and the label.

In [4]:
# Create parser
# Args: filenames
# Returns: tensor containing read and decoded element
def _parse_data(filename0, filename1, label):
    # Read and decode first file
    matrix0 = tf.read_file(filename0)
    matrix0 = tf.decode_raw(matrix0, out_type=tf.float32)
    matrix0 = tf.reshape(matrix0, [height,width])
    
    # Read and decode second file
    matrix1 = tf.read_file(filename1)
    matrix1 = tf.decode_raw(matrix1, out_type=tf.float32)
    matrix1 = tf.reshape(matrix1, [height,width])
    
    # Stack the two elements together
    X = tf.stack([matrix0, matrix1])
    
    # Get label (you could implement more complex logic here if needed)
    y = tf.reshape(label, [1])
    
    return X, y

### Create data augmentation function

We will need a function to implement the data augmentation logic.

This function takes one example (X,y), and returns a dataset with two examples: the original example, and a second one generated on the fly.

In [5]:
# Data augmentation function: create several examples from one example
# Args: One example X,y
# Returns: Dataset containing several examples, after data augmentation
def _data_augment(X,y):
    # Generate new data from example X
    X0 = X
    X1 = -X # Dummy data augmentation, but we could use any transformation we need. We could also generate more examples.
    X_augmented = tf.stack([X0,X1])
    
    # Repeat y
    y_augmented = tf.stack([y,y])
    
    dataset = tfdata.Dataset.from_tensor_slices((X_augmented, y_augmented))
        
    return dataset

### It's almost done

Now that we have implemented our parsing and data augmentation logic, we can create the dataset.

As you can see, with the TensorFlow Dataset API, it is really easy! 

In [6]:
####################################################################
# Create dataset from files
####################################################################

# Get the filenames lists
filenames0 = ['data/file_0_0.raw', 'data/file_0_1.raw']
filenames1 = ['data/file_1_0.raw', 'data/file_1_1.raw']

# Create tensorflow constant containing the filenames
tf_filenames0 = tf.constant(filenames0)
tf_filenames1 = tf.constant(filenames1)

# Create labels dataset
labels = tf.constant([15, 25])

# Create dataset containing filenames and labels
dataset = tfdata.Dataset.from_tensor_slices((tf_filenames0, tf_filenames1, labels))

# Use our _parse_data function to read files and decode data
dataset = dataset.map(_parse_data)

#####################################################################
# Data augmentation
#####################################################################

# Apply data augmentation
dataset = dataset.interleave(_data_augment, cycle_length=1)

### Print the dataset

We can now print the dataset to check that everything is working.

In [7]:
#####################################################################
# Print the dataset
#####################################################################

# create TensorFlow Iterator object
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:

    # iterate over the dataset
    while True:
        try:
            elem = sess.run(next_element)
            print(elem)
        except tf.errors.OutOfRangeError:
            print("End of dataset")
            break

(array([[[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.]]], dtype=float32), array([15]))
(array([[[-0., -0., -0.],
        [-0., -0., -0.],
        [-0., -0., -0.]],

       [[-1., -1., -1.],
        [-1., -1., -1.],
        [-1., -1., -1.]]], dtype=float32), array([15]))
(array([[[ 10.,  10.,  10.],
        [ 10.,  10.,  10.],
        [ 10.,  10.,  10.]],

       [[ 11.,  11.,  11.],
        [ 11.,  11.,  11.],
        [ 11.,  11.,  11.]]], dtype=float32), array([25]))
(array([[[-10., -10., -10.],
        [-10., -10., -10.],
        [-10., -10., -10.]],

       [[-11., -11., -11.],
        [-11., -11., -11.],
        [-11., -11., -11.]]], dtype=float32), array([25]))
End of dataset


### It is working!

We can see that parsing and data augmentation work well:

* each example contains:
  * two matrices coming from two different files
  * a label
* after each example, there is an augmented example (-X, y)

## Let's train our model now

### Create a model
First we create a small dummy model.

In [8]:
# Create a small dummy model
def model_function(input_data):
    flattened = tf.contrib.layers.flatten(input_data)
    a1 = tf.layers.dense(flattened, 100, activation=tf.nn.relu)
    output = tf.layers.dense(a1, 1)
    return output

# Create the loss function    
def loss_function(prediction, label):
    return tf.losses.mean_squared_error(prediction, label)

Now we will do the following:

* Prepare the dataset for training
  * Repeat the dataset (remove this in real application, we do it only because in this example we have too few data)
  * Set the batch size
  * Create an feedable iterator on the batched dataset. The feedable iterator will allow us to switch between training and validation dataset, see documentation here: https://www.tensorflow.org/programmers_guide/datasets
* Create a global_step variable. This variable will count the number of training steps. It will be saved with the model, and will allow us to resume training.
* Create the predict op, loss op and train op.

In [9]:
# Repeat dataset in order to simulate more data (remove this in real application)
batch_dataset = dataset.repeat(30)

# Set batch size
batch_dataset = batch_dataset.batch(2)

# create TensorFlow Iterator object
handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(
    handle, batch_dataset.output_types, batch_dataset.output_shapes)

# Create training_iterator
training_iterator = batch_dataset.make_initializable_iterator()

# Get example, compute loss and create optimizer
next_batch_examples, next_batch_labels = iterator.get_next()

# Create global_step variable, to store the global training step
global_step = tf.Variable(0, name='global_step', trainable=False)

# Create ops that we will use during training, loss computation and prediction
predict_op = model_function(next_batch_examples)
loss_op = loss_function(predict_op, next_batch_labels)
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss_op, global_step=global_step)

### That's all. We can train!

We start a training session. The training process is quite standard, see https://www.tensorflow.org/get_started/mnist/mechanics for more details.

In this example, we will also print training loss periodically.

In [10]:
# Train

number_of_epochs = 3

with tf.Session() as sess:
    
    # Initialize variables
    init = tf.global_variables_initializer()
    sess.run(init)
    
    # Feed iterator with training data
    training_handle = sess.run(training_iterator.string_handle())
       
    for epoch in range(number_of_epochs):
        
        # Tell iterator to go to beginning of dataset
        sess.run(training_iterator.initializer)

        # Initialize variables
        losses = []
        iteration = 0
        
        print ("Starting epoch: ", epoch)
        
        # iterate over the training dataset and train
        while True:
            try:
                loss,_ = sess.run([loss_op, train_op], feed_dict={handle: training_handle})
                losses.append(loss)
                if iteration % 25 == 0:                  
                    print("Training loss at iteration %i: %f " % (iteration, sum(losses)/len(losses)))
                iteration += 1  
            except tf.errors.OutOfRangeError:
                print("--------------")
                break
                
    print("Total number of training steps: ", sess.run(global_step))
    
    

Starting epoch:  0
Training loss at iteration 0: 237.014236 
Training loss at iteration 25: 339.300055 
Training loss at iteration 50: 240.801999 
--------------
Starting epoch:  1
Training loss at iteration 0: 187.896851 
Training loss at iteration 25: 92.969591 
Training loss at iteration 50: 92.489890 
--------------
Starting epoch:  2
Training loss at iteration 0: 171.550018 
Training loss at iteration 25: 84.646680 
Training loss at iteration 50: 84.787011 
--------------
Total number of training steps:  180


## Validation cost during training

Now, let's see how we can enhance the previous code to compute validation loss periodically during training.

### Create validation dataset

I will not comment this part. It is just about creating a fake validation dataset. You should replace this to create your own real validation dataset.

In [11]:
# Create random files containing raw matrix of shape 3x3
for i in range(2):
    for j in range(2):
        # Create matrix with fake data
        matrix = np.zeros((height,width)) + i + (j*10) + 2
        # Save matrix as raw float32 file
        matrix.astype('float32').tofile('data/validation_' + str(i) + '_' + str(j) + '.raw')
        
# Get the filenames lists
val_filenames0 = ['data/validation_0_0.raw', 'data/validation_0_1.raw']
val_filenames1 = ['data/validation_1_0.raw', 'data/validation_1_1.raw']

# Create tensorflow constant containing the filenames
tf_val_filenames0 = tf.constant(val_filenames0)
tf_val_filenames1 = tf.constant(val_filenames1)

# Create labels dataset
val_labels = tf.constant([19, 21])

# Create dataset containing filenames and labels
val_dataset = tfdata.Dataset.from_tensor_slices((tf_val_filenames0, tf_val_filenames1, labels))

# Use our _parse_data function to read files and decode data
val_dataset = val_dataset.map(_parse_data)

# Set batch size
val_dataset = val_dataset.batch(1)

# create TensorFlow Iterator object
val_iterator = val_dataset.make_initializable_iterator()

### Compute validation loss

We define a *run_validation* function. This function iterates over a validation dataset, computes losses for each batch, and returns the mean of the losses.

Many loss functions can be aggregated by averaging losses over batches. But for some loss functions, you might have to change the way you make the aggregation. For instance, for RMSE loss, you would have to square the losses before taking the average.

In [12]:
def run_validation(loss_op, my_handle, sess):  # Called inside train_loop()
    # iterate over the dataset
    losses = []
    while True:
        try:
            losses.append(sess.run(loss_op, feed_dict={handle: my_handle}))
        except tf.errors.OutOfRangeError:
            break
    
    # Return the mean of the losses
    return sum(losses) / len(losses)

### Training!

The training process is similar to above. We have added computation of validation loss at every epoch.

In [13]:
# Train and validate

number_of_epochs = 3

with tf.Session() as sess:
    
    # Initialize variables
    init = tf.global_variables_initializer()
    sess.run(init)
    
    # Create training data and validation data handles
    training_handle = sess.run(training_iterator.string_handle())
    validation_handle = sess.run(val_iterator.string_handle())
    
    for epoch in range(number_of_epochs):
        
        # Tell iterator to go to beginning of dataset
        sess.run(training_iterator.initializer)
    
        # Initialize variables
        
        losses = []
        iteration = 0
        
        print ("Starting epoch: ", epoch)
    
        # iterate over the training dataset and train
        while True:
            try:
                loss,_ = sess.run([loss_op, train_op], feed_dict={handle: training_handle})
                losses.append(loss)
                if iteration % 25 == 0:                  
                    print("Training loss at iteration %i: %f " % (iteration, sum(losses)/len(losses)))
                iteration += 1                    
            except tf.errors.OutOfRangeError:
                print("***** Summary for epoch %i - %i iterations *****" % (epoch, iteration))
                print("Training loss: %f " % (sum(losses)/len(losses)))
                break              

        # Tell validation iterator to go to beginning of dataset
        sess.run(val_iterator.initializer)

        # run validation
        print("Validation loss: ", run_validation(loss_op, validation_handle, sess))
        print("---------------")
        
    print("Total number of training steps: ", sess.run(global_step))
    

Starting epoch:  0
Training loss at iteration 0: 221.550812 
Training loss at iteration 25: 207.128792 
Training loss at iteration 50: 155.190418 
***** Summary for epoch 0 - 60 iterations *****
Training loss: 144.457278 
Validation loss:  52.0583219528
---------------
Starting epoch:  1
Training loss at iteration 0: 184.710526 
Training loss at iteration 25: 91.274975 
Training loss at iteration 50: 91.411974 
***** Summary for epoch 1 - 60 iterations *****
Training loss: 89.157889 
Validation loss:  47.2030735016
---------------
Starting epoch:  2
Training loss at iteration 0: 170.304825 
Training loss at iteration 25: 83.665898 
Training loss at iteration 50: 83.477095 
***** Summary for epoch 2 - 60 iterations *****
Training loss: 81.309501 
Validation loss:  42.9728765488
---------------
Total number of training steps:  180


## Monitor training with TensorBoard

If you want to train your model accurately, you will need to monitor the training process.

TensorFlow provides a nice web interface for monitoring, called TensorBoard. See here: https://www.tensorflow.org/get_started/summaries_and_tensorboard

We are going to add a few lines in our previous code, in order to write logs for TensorBoard. We will log the following data:
* Training loss every 25 steps
* Average of training loss over each epoch
* Average of validation loss after each epoch

The logs will be written to `./logs` directory. To launch TensorBoard and visualize logs, open a terminal, cd to this notebook directory, and enter:
```bash
tensorboard --logdir=logs
```

In [14]:
# Train, validate and write logs for TensorBoard

number_of_epochs = 3

with tf.Session() as sess:
    
    # Initialize variables
    init = tf.global_variables_initializer()
    sess.run(init)
    
    # Create training data and validation data handles
    training_handle = sess.run(training_iterator.string_handle())
    validation_handle = sess.run(val_iterator.string_handle())
    
    # Logs for TensorBoard: create summary writers
    train_writer = tf.summary.FileWriter('logs/train',
                                      sess.graph)
    val_writer = tf.summary.FileWriter('logs/val')

    
    # Create a summary to monitor loss
    tf.summary.scalar("loss", loss_op)
    # In this example, we have only one summary op, but if you have many, it is convenient to merge them in one op
    merged_summary_op = tf.summary.merge_all()
    
    for epoch in range(number_of_epochs):
        
        # Tell iterator to go to beginning of dataset
        sess.run(training_iterator.initializer)
    
        # Initialize variables
        
        losses = []
        iteration = 0
        
        print ("Starting epoch: ", epoch)
    
        # iterate over the training dataset and train
        while True:
            try:
                global_iteration, merged_summary, loss,_ = sess.run([global_step, merged_summary_op, loss_op, train_op], feed_dict={handle: training_handle})
                losses.append(loss)
                if iteration % 25 == 0:                  
                    print("Training loss at iteration %i: %f " % (iteration, sum(losses)/len(losses)))
                    # Log loss for tensorboard
                    train_writer.add_summary(merged_summary, global_iteration)
                iteration += 1
            except tf.errors.OutOfRangeError:
                print("***** Summary for epoch %i - %i iterations *****" % (epoch, iteration))
                loss_over_epoch = sum(losses)/len(losses)
                print("Training loss: %f " % loss_over_epoch)
                
                # Log average loss for this epoch (for TensorBoard)
                avg_loss_summary = tf.Summary()
                avg_loss_summary.value.add(tag="epochLoss", simple_value=loss_over_epoch)
                train_writer.add_summary(avg_loss_summary, global_iteration)                
                break              

        # Tell validation iterator to go to beginning of dataset
        sess.run(val_iterator.initializer)

        # run validation
        validation_loss = run_validation(loss_op, validation_handle, sess)
        print("Validation loss: ", validation_loss)
        print("---------------")
        
        # Log validation loss for TensorBoard
        val_loss_summary = tf.Summary()
        val_loss_summary.value.add(tag="valLoss", simple_value=validation_loss)
        val_writer.add_summary(val_loss_summary, global_iteration)
        
    print("Total number of training steps: ", sess.run(global_step))
    

Starting epoch:  0
Training loss at iteration 0: 226.079163 
Training loss at iteration 25: 284.721069 
Training loss at iteration 50: 201.070693 
***** Summary for epoch 0 - 60 iterations *****
Training loss: 183.505833 
Validation loss:  54.5265464783
---------------
Starting epoch:  1
Training loss at iteration 0: 185.984634 
Training loss at iteration 25: 92.209653 
Training loss at iteration 50: 92.439193 
***** Summary for epoch 1 - 60 iterations *****
Training loss: 90.206898 
Validation loss:  48.236114502
---------------
Starting epoch:  2
Training loss at iteration 0: 173.080841 
Training loss at iteration 25: 85.272181 
Training loss at iteration 50: 85.383511 
***** Summary for epoch 2 - 60 iterations *****
Training loss: 83.264335 
Validation loss:  44.6632089615
---------------
Total number of training steps:  180


## Saving and restoring the model
Training a model can take a lot of time. It is wise to save periodically the result of your training, to be able to restore it whenever you need.

This is what we are going to add to our code:
* Before we start training, check if we already have a saved model. If so, load it.
* During training, save the model periodically.
* At the end of training, save the model.

In [15]:
# Train, validate and write logs for TensorBoard

number_of_epochs = 3

saving_dir = 'saved_model'

# Add ops to save and restore the model
saver = tf.train.Saver(max_to_keep=20)

with tf.Session() as sess:
    
    # Check if model has been previously saved
    if os.path.isfile(saving_dir + '/checkpoint'):
        # Restore model from file
        saver.restore(sess, tf.train.latest_checkpoint(saving_dir))
    else:
        # Initialize variables
        init = tf.global_variables_initializer()
        sess.run(init)
    
    # Restore the previously saved model
    
    # Create training data and validation data handles
    training_handle = sess.run(training_iterator.string_handle())
    validation_handle = sess.run(val_iterator.string_handle())
    
    # Logs for TensorBoard: create summary writers
    train_writer = tf.summary.FileWriter('logs/train',
                                      sess.graph)
    val_writer = tf.summary.FileWriter('logs/val')

    
    # Create a summary to monitor loss
    tf.summary.scalar("loss", loss_op)
    # In this example, we have only one summary op, but if you have many, it is convenient to merge them in one op
    merged_summary_op = tf.summary.merge_all()
    
    for epoch in range(number_of_epochs):
        
        # Tell iterator to go to beginning of dataset
        sess.run(training_iterator.initializer)
    
        # Initialize variables
        
        losses = []
        iteration = 0
        
        print ("Starting epoch: ", epoch)
    
        # iterate over the training dataset and train
        while True:
            try:
                global_iteration, merged_summary, loss,_ = sess.run([global_step, merged_summary_op, loss_op, train_op], feed_dict={handle: training_handle})
                losses.append(loss)
                if iteration % 25 == 0:                  
                    print("Training loss at iteration %i: %f " % (iteration, sum(losses)/len(losses)))
                    # Log loss for tensorboard
                    train_writer.add_summary(merged_summary, global_iteration)
                iteration += 1

                # Save model periodically
                if global_iteration % 100 == 0:
                    save_path = saver.save(sess, saving_dir + "/model.ckpt", global_step=global_iteration)
                    print("-----> Model saved in file: %s" % save_path)

            except tf.errors.OutOfRangeError:
                print("***** Summary for epoch %i - %i iterations *****" % (epoch, iteration))
                loss_over_epoch = sum(losses)/len(losses)
                print("Training loss: %f " % loss_over_epoch)
                
                # Log average loss for this epoch (for TensorBoard)
                avg_loss_summary = tf.Summary()
                avg_loss_summary.value.add(tag="epochLoss", simple_value=loss_over_epoch)
                train_writer.add_summary(avg_loss_summary, global_iteration)                
                break              

        # Tell validation iterator to go to beginning of dataset
        sess.run(val_iterator.initializer)

        # run validation
        validation_loss = run_validation(loss_op, validation_handle, sess)
        print("Validation loss: ", validation_loss)
        print("---------------")
        
        # Log validation loss for TensorBoard
        val_loss_summary = tf.Summary()
        val_loss_summary.value.add(tag="valLoss", simple_value=validation_loss)
        val_writer.add_summary(val_loss_summary, global_iteration)

    print("Total number of training steps: ", sess.run(global_step))
    
    # Save the model at end of training
    save_path = saver.save(sess, saving_dir + "/model.ckpt", global_step=sess.run(global_step))
    print("-----> Model saved in file: %s" % save_path)
    

Starting epoch:  0
Training loss at iteration 0: 214.183395 
Training loss at iteration 25: 164.949371 
Training loss at iteration 50: 130.911270 
***** Summary for epoch 0 - 60 iterations *****
Training loss: 123.107073 
Validation loss:  46.0443506241
---------------
Starting epoch:  1
Training loss at iteration 0: 173.353851 
Training loss at iteration 25: 85.321146 
-----> Model saved in file: saved_model/model.ckpt-100
Training loss at iteration 50: 84.797553 
***** Summary for epoch 1 - 60 iterations *****
Training loss: 82.510118 
Validation loss:  41.0695266724
---------------
Starting epoch:  2
Training loss at iteration 0: 154.391617 
Training loss at iteration 25: 75.326975 
Training loss at iteration 50: 74.429415 
***** Summary for epoch 2 - 60 iterations *****
Training loss: 72.268382 
Validation loss:  34.3418731689
---------------
Total number of training steps:  180
-----> Model saved in file: saved_model/model.ckpt-180


## Make a prediction

Now we have a trained model, we can make predictions.

### Create input dataset

As we have used an iterator to feed our model, we need to put our examples in an iterator

In [16]:
# For illustration purpose, we will take our validation examples to feed the predictor.
# In real life, you will of course use some other data.

# Get the filenames lists
input_filenames0 = ['data/validation_0_0.raw', 'data/validation_0_1.raw']
input_filenames1 = ['data/validation_1_0.raw', 'data/validation_1_1.raw']

# Create tensorflow constant containing the filenames
tf_input_filenames0 = tf.constant(input_filenames0)
tf_input_filenames1 = tf.constant(input_filenames1)

# Create labels. These data will not be used, but we need its type and shape to match the training dataset.
predict_labels = tf.constant([0,0])

# Create dataset containing filenames and labels
input_dataset = tfdata.Dataset.from_tensor_slices((tf_input_filenames0, tf_input_filenames1, predict_labels))

# Use our _parse_data function to read files and decode data
input_dataset = input_dataset.map(_parse_data)

# Set batch size
input_dataset = input_dataset.batch(1)

# create TensorFlow Iterator object
input_iterator = input_dataset.make_initializable_iterator()

### Make the prediction

Here, we will load the model from file, and then feed our iterator into the model to make the prediction.

In [17]:
saving_dir = 'saved_model'

# Add ops to save and restore the model
saver = tf.train.Saver()

with tf.Session() as sess:
        
    # Create iterator handle with input data
    input_handle = sess.run(input_iterator.string_handle())
       
    # Check if model has been previously saved
    if os.path.isfile(saving_dir + '/checkpoint'):
        # Restore model
        saver.restore(sess, tf.train.latest_checkpoint(saving_dir))
    else:
        print("Unable to find model file.")

    # Initialize a list that will contain our predictions
    predictions=[]
        
    # Tell iterator to go to beginning of dataset
    sess.run(input_iterator.initializer)

    i = 0

    # iterate over the input dataset and make prediction
    while True:
        try:
            prediction = sess.run([predict_op], feed_dict={handle: input_handle})
            predictions.append(prediction)
            print("Prediction %i: " %i , prediction[0][0][0])
            i += 1
        except tf.errors.OutOfRangeError:
            break
                
    print("We have made %i predictions." % (len(predictions)))

INFO:tensorflow:Restoring parameters from saved_model\model.ckpt-180
Prediction 0:  8.2288
Prediction 1:  29.7786
We have made 2 predictions.


## License

This notebook is released under the Apache 2.0 license.

https://www.apache.org/licenses/LICENSE-2.0

Copyright © `2017` `Lior Perez`